In [1]:
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI, ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.graphs import Neo4jGraph
from operator import itemgetter
# Load environment variables from .env file
from functools import wraps
import time
import os
import time  
load_dotenv()

AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_KEY")


# llm = AzureChatOpenAI(
#     azure_endpoint=AZURE_OPENAI_ENDPOINT,
#     openai_api_version='2023-03-15-preview',
#     azure_deployment='gpt-4',
#     openai_api_key=AZURE_OPENAI_API_KEY,
# )

llm = ChatOpenAI(
    api_key='sk-proj-gfA9WQMdhwOmo2lOMFtAT3BlbkFJ9qwC52ExKO4FTTatmUNH',
    model='gpt-4o', 
)


def FuncTimeRet(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        ts = time.perf_counter()
        ret = func(*args, **kwargs)
        te = time.perf_counter()
        print(f'>>-- {func.__name__}: takes {te - ts:0.4f} seconds --<<')
        return ret
    return wrapper

In [2]:
llm.invoke('hi')

AIMessage(content='Hello! How can I assist you today?')

In [2]:
from docx2pdf import convert
import os

# Define the path to the Word document
word_file_path = r"C:\Users\Ai Sukmoren\OneDrive\project research\The Impact of Discrimination in the Workplace-DESKTOP-UB36V5M.docx"

# Define the output directory (Desktop)
output_directory = os.path.join(os.path.expanduser("~"), "Desktop")

# Convert the Word document to PDF
convert(word_file_path, output_directory)

print(f"PDF saved to {output_directory}")


c:\Users\Ai Sukmoren\Desktop\kmitl-chatbot\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1/1 [00:07<00:00,  7.25s/it]

PDF saved to C:\Users\Ai Sukmoren\Desktop
